# Read in the acceleration mask for k-space and determine the accelration factor for R=3 and R=6

In [3]:
from pathlib import Path
import SimpleITK as sitk


mask_r6_fpath = Path('/scratch/hb-pca-rad/projects/04_uncertainty_quantification/gaussian/recons_6x/0003_ANON5046358/VSharpNet_R6_mask.nii.gz')

In [ ]:
# --- Step 1: Load the Sampling Mask ---
# Read the k-space mask (which indicates which columns are measured)
mask_r6 = sitk.ReadImage(str(mask_r6_fpath))
mask_r6_arr = sitk.GetArrayFromImage(mask_r6)
print(f"Shape of the mask: {mask_r6_arr.shape}")  # Expected shape: (slices, rows, cols)

# --- Step 2: Determine Measured Columns ---
# We count the number of non-zero columns (i.e., columns with any measured data)
# "any(axis=(0, 1))" collapses slices and rows, leaving a boolean for each column.
measured_cols = mask_r6_arr.any(axis=(0, 1)).sum()
print(f"Number of non-zero columns: {measured_cols}")

# Total number of columns (full k-space grid)
all_lines = mask_r6_arr.shape[-1]

# --- Step 3: Compute the Empirical Acceleration Factor ---
# If fully sampled, all columns would be measured; hence, the ratio all_lines/measured_cols
acc_factor = all_lines / measured_cols
print(f"Acceleration factor (mask ratio): {acc_factor}")

# --- Step 4: Incorporate the Acquisition Strategy ---
# Our TSE sequence has 3 averages, but we use only 1, so the base acceleration is 3.
base_acc_factor = 3

# Multiply the base factor by the measured acceleration factor.
final_acc = base_acc_factor * acc_factor
print(f"Acceleration factor after combining with base factor: {final_acc}")

# --- Step 5: Adjust for Partial Sampling per Average ---
# Each average only acquires either odd or even lines,
# so we divide by 2 to reflect that only half of the lines are actually measured in an average.
final_acc /= 2
print(f"Final acceleration factor after adjustment: {final_acc}")


Shape of the mask: (30, 768, 651)
Number of non-zero columns: 163
Acceleration factor (mask ratio): 3.9938650306748467
Acceleration factor after combining with base factor: 11.98159509202454
Final acceleration factor after adjustment: 5.99079754601227
